# Step 6 - Plotting bicycle network analysis results
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the results from 03_poi_based_generation and 04_connect_clusters and 05_analyze_results, and plots them. The plots are not yet polished.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-21

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
bbox = {}
bbox["inline"] = (250,250)
bbox["export"] = (800,800)

%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Plot

Some plot functions love to plot inline. To suppress that and only export the plots, we use ``%%capture``.  
To check that plots are being generated while running the code below, go into the ``plots/placeid`` folder.

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting results")
    
    # EXISTING INFRASTRUCTURE
    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    
    # PLOT Maps
    my_plot_reset(G_carall, nnids)
    ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_carall_poi_' + poi_source + ".png", bbox = bbox["export"])
    my_plot_reset(G_carall, [])
    ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_carall.png', bbox=bbox["export"])
    my_plot_reset(G_biketrack, [])
    ig.plot(G_biketrack, PATH["plots"] + placeid + "/" + placeid + '_biketrack.png', bbox = bbox["export"])
    my_plot_reset(G_bikeable, [])
    ig.plot(G_bikeable, PATH["plots"] + placeid + "/" + placeid + '_bikeable.png', bbox = bbox["export"])
    my_plot_reset(G_biketrackcarall, [])
    ig.plot(G_biketrackcarall, PATH["plots"] + placeid + "/" + placeid + '_biketrackcarall.png', bbox = bbox["export"])
    # PLOT Analysis (to do)
    
    
    
    # GENERATED, POI BASED
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    if debug: pp.pprint(res)
    
    # PLOT abstract greedy triangulation
    for GT_abstract, prune_quantile in zip(res["GT_abstracts"], res["prune_quantiles"]):
        color_nodes(GT_abstract, "red", False, False)
        size_nodes(GT_abstract, 6, False, False)
        ig.plot(GT_abstract, PATH["plots"] + placeid + "/" + placeid + '_GTabstract_poi_'+ poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox["export"])
        
    # PLOT just the bicycle network
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        size_nodes(GT, 0)
        width_edges(GT, 2)
        color_edges(GT, "blue")
        ig.plot(GT, PATH["plots"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox["export"])
        
    # PLOT all together (this can take at least a minute per loop)
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        my_plot_reset(G_carall, nnids)
        color_nodes(G_carall, "red", nnids)
        size_nodes(G_carall, 6, nnids)
        GT_ids = [v["id"] for v in GT.vs]  
        color_nodes(G_carall, "blue", GT_ids)
        size_nodes(G_carall, 5, GT_ids)
        color_nodes(G_carall, "cyan", list(set(GT_ids).intersection(set(nnids))))
        ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_GTall_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox["export"])
        
    # PLOT Analysis (to do)
    
    
    
    # GENERATED, CLUSTER BASED
    # Load results
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    if debug: pp.pprint(res)
        
    # PLOT abstract greedy triangulation
    for GT_abstract, prune_quantile in zip(res["GT_abstracts"], res["prune_quantiles"]):
        color_nodes(GT_abstract, "red", False, False)
        size_nodes(GT_abstract, 6, False, False)
        ig.plot(GT_abstract, PATH["plots"] + placeid + "/" + placeid + '_GTclustersabstract_biketrack_cutoff' + cutofftype + "{:.2f}".format(cutoff) + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox['export'])
    
    # PLOT clusters with their new connections (last prune_measure only)
    plt.figure(figsize=[2*6.4, 2*4.8])
    plt.plot(res["GTs"][-1].vs['x'], res["GTs"][-1].vs['y'], 'o', color='red', markersize=5)
    for i in range(len(res["clusters"])):
        plt.plot(res["clusters"][i].vs['x'], res["clusters"][i].vs['y'], 'o', markersize=3)
    plt.gca().invert_yaxis()
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTclustersonly_biketrack_cutoff' + cutofftype + "{:.2f}".format(cutoff) + "_" + prune_measures[prune_measure] + "{:.3f}".format(res["prune_quantiles"][-1]) + '.png')
    %config InlineBackend.figure_format = 'retina'
    
    # PLOT the whole bike network and the new connections
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        plt.figure(figsize=[2*6.4, 2*4.8])
        plt.plot(GT.vs['x'], GT.vs['y'], 'o', color='red', markersize=3)
        plt.plot(G_biketrack.vs['x'], G_biketrack.vs['y'], 'o', color='black', markersize=1)
        plt.gca().invert_yaxis()
        %config InlineBackend.figure_format = 'retina'
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTclusters_biketrack_cutoff' + cutofftype + "{:.2f}".format(cutoff) + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png')
        
    # PLOT Analysis (to do)

Possibly look into folium:  
https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf  
https://github.com/socib/Leaflet.TimeDimension

In [ ]:
Audio(sound_file, autoplay=True)